# Suivi de trajectoire - Entrainement du modèle

Dans ce notebook, nous allons entrainer le modèle qui prend en entrée une image et nous donne en sortie les coordonnées X,Y de la cible.

Nous allons utiliser le modèle Resnet18 avec Keras / Tensorflow.

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import shutil

from tensorflow import keras
from matplotlib import pyplot as plt

### Téléchargement et extraction des données collectées 

Tout d'abord il faut récupérer les données collectées contenues dans le fichier ``dataset.zip`` que nous avons créé précedemment. Pour cela, il suffit d'exécuter la commande ci-dessous.

In [ ]:
#!unzip -q dataset.zip

Un dossier nommé ``dataset_xy`` devrait apparaître dans l'explorateur de fichiers.

# Création du dataset

### 1. Récupération du chemin des images

In [ ]:
import glob

repertoires_images = []
repertoire_courant = os.getcwd()
for fichier in glob.glob(repertoire_courant+"/dataset_xy/*.jpg"):
    repertoires_images.append(fichier)

In [ ]:
repertoires_images

### 2. Création du dataset d'entrainement en type regression

On va créer le dataset en ajoutant à chaque entrée un label correspondant. Ce label sera les coordonnées X,Y extraites à partir du nom du fichier.

In [ ]:
# Fonction de création du dataset d'entrainement
# à partir de la liste des fichiers

# /home/alexandre/Notebook/Jetbot/Suivi_trajectoire/dataset_xy/xy_060_174_5dd5ae38-b8bc-11ec-a2fb-401c8381bcee.jpg

def CreationDatasetRegression(liste_fichiers,width=224, height=224):
    # Listes dans lesquelles ont va sauvegarder les images et les labels
    images_= []
    coordonnees_ = []

    for fichier in liste_fichiers:
        # Extraction des coordonnées (x0,Y0)
        # correspondantes aux dimensions de l'image chargée (height,width)
        element = tf.strings.split(fichier,sep="xy_")
        element = tf.strings.split(element[1],sep="_")
        x0 = (tf.strings.to_number(element[0],out_type=tf.dtypes.float32) - width/2) / (width/2)
        y0 = (tf.strings.to_number(element[1],out_type=tf.dtypes.float32) - height/2) / (height/2)

        # Chargement de l'image
        image = tf.keras.preprocessing.image.load_img(fichier)
        
        # Transformations éventuelles de l'image
        image = tf.image.random_saturation(tf.keras.preprocessing.image.img_to_array(image), 0.7, 1.3)
        image = tf.image.random_hue(image, 0.3)
        image = tf.image.random_brightness(image, 0.7, 1.3)
        image = tf.image.random_contrast(image, 0.7, 1.3)

        # Sauvegarde de l'image et des coordonnées dans les listes
        images_.append(tf.cast(tf.keras.preprocessing.image.img_to_array(image),tf.uint8))
        coordonnees_.append(np.asarray([x0,y0]))
        
        # Ajout d'une image inversée horizontalement
        image = tf.image.flip_left_right(tf.keras.preprocessing.image.img_to_array(image))
        x0 = -x0
        images_.append(tf.cast(image,tf.uint8))
        coordonnees_.append(np.asarray([x0,y0]))
        
    # Création du dataset
    images_ = tf.convert_to_tensor(images_)                                # (nbr_images,224,224,3)
    coordonnees_ = tf.convert_to_tensor(coordonnees_)                      # (nbr_images,2)

    datasetImages = tf.data.Dataset.from_tensors(images_)                  # (nbr_images,224,224,3)
    datasetCoordonnees = tf.data.Dataset.from_tensors(coordonnees_)        # (nbr_images,2) 
    dataset = tf.data.Dataset.zip((datasetImages,datasetCoordonnees))

    return (dataset)

In [ ]:
dataset_regression = CreationDatasetRegression(repertoires_images,224,224)

Regardons à quoi ressemble le dataset :

In [ ]:
for image,coordonnees in dataset_regression.take(1):
    print(image.shape)
    print(coordonnees.shape)

In [ ]:
for image,coordonnees in dataset_regression.take(1):
    i = np.random.randint(0,len(repertoires_images)*2)
    xs = np.int(224 * ((coordonnees[i,0]+1) / 2.0))
    ys = np.int(224 * ((coordonnees[i,1]+1) / 2.0))

    image = cv2.circle(np.array(image[i,:,:,:]), (xs,ys), 8, (255, 0, 0), 3)
    image = cv2.line(np.array(image),(112,224),(xs,ys),(0,0,255),3)
    plt.imshow(image)

On redimensionne le dataset au bon batch_size :

In [ ]:
batch_size = 5
dataset_regression = dataset_regression.unbatch()
dataset_regression = dataset_regression.batch(batch_size)

In [ ]:
for image,coordonnees in dataset_regression.take(1):
    print(image.shape)
    print(coordonnees.shape)

### Séparation du dataset en dataset d'entrainement et de validation
On réserve 90% des images pour l'entrainement et 10% pour les validations.

In [ ]:
pourcentage_entrainement = 0.8

# Récupère la taille du dataset
taille_dataset = len(list(dataset_regression))

# Calcul des tailles d'entrainement et de validation
taille_entrainement = int(pourcentage_entrainement * taille_dataset)
taille_validation = int((1-pourcentage_entrainement) * taille_dataset)

# Mélange du dataset
dataset_regression = dataset_regression.shuffle(taille_dataset)

# Création des dataset d'entrainement et de validation
dataset_entrainement = dataset_regression.take(taille_entrainement)
dataset_validation = dataset_regression.take(taille_validation)

# Création du modèle Restnet-18 

In [ ]:
#!pip install git+https://github.com/qubvel/classification_models.git

In [ ]:
import classification_models
from classification_models.tfkeras import Classifiers

# Chargement du modèle ResNEt18
ResNet18, preprocess_input = Classifiers.get('resnet18')

# Instanciation du modèle pré-entrainé ResNet18
base_model = ResNet18(input_shape=(224,224,3), weights='imagenet', include_top=False, pooling=False)

model = tf.keras.models.Model(inputs=base_model.input, outputs=base_model.output)
model.summary()

In [ ]:
# Désactivation des couches pour l'entrainement
#for layer in base_model.layers:
#    layer.trainable = False

On ajoute ensuite la couche d'applatissemnt des sorties et la couche dense avec 2 neurones (1 neurone par coordonée) :

In [ ]:
# Ajout de l'applatissement des sorties et de la couche dense avec 2 neurones"
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(units=2, activation="linear")(x)

# Création du modèle global
model = tf.keras.Model(inputs=[base_model.input], outputs=[output])

# Affichage des informations sur le modèle
model.summary()

### Entrainement en régression:

On entraine le modèle sur 100 périodes. Cette fois on utilise l'erreur MSE (Mean Square Error).

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Nombre de périodes d'entrainement
periodes = 100

# Définition de la fonction d'enregistrement automatique du meilleur modèle
model_save = ModelCheckpoint('meilleur_modele.hdf5', save_best_only=True, monitor='val_loss', mode='min')

# Définition de l'ooptimiseur
adam = keras.optimizers.Adam(learning_rate=1e-3)

# Entrainement du modèle : from_logits=True car on utilise un Softmax en sortie de notre modèle
model.compile(optimizer=adam, loss='mse')
historique = model.fit(dataset_entrainement,validation_data=dataset_validation,verbose=1,epochs=periodes, callbacks=[model_save])

In [ ]:
loss = historique.history['loss']
val_loss = historique.history['val_loss']

intervalle_periodes = range(periodes)

plt.plot(intervalle_periodes, loss, label="Erreur d'entrainement")
plt.plot(intervalle_periodes, val_loss, label="Erreur de validation")
plt.legend(loc='upper right')
plt.title("Erreur d'entrainement et de validation")
plt.show()